# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py F1AP.docx 'keyword'" for F1AP and "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/spec_ingestor.py NGAP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: UE Context established after initial failure when a UE Context Setup Success message follows an incorrect UE Context Setup Request.
Repro Steps:
1) Attacker sends an incorrect UE Context Setup Request to the gNB-CU.
2) gNB-CU rejects the request as expected.
3) Attacker immediately sends a UE Context Setup Success.
4) In parallel, attacker sends an initialRRCTransfer with mutated DU ID and Transaction ID.
5) Attacker follows with an RRC Setup Success message.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

```json
{
  "bug_card": {
    "observed_behavior": "UE Context established despite initial failure. A UE Context Setup Success message follows an incorrect UE Context Setup Request (which was rejected), accompanied by a parallel Initial RRC Transfer with mutated IDs, leading to RRC Setup Success.",
    "interface_guess": [
      "F1AP"
    ],
    "procedure_guess": [
      "UE Context Setup",
      "Initial UL RRC Message Transfer"
    ],
    "components_involved": [
      "gNB-CU",
      "gNB-DU"
    ],
    "key_ids": {
      "transaction_id": "mutated in attack",
      "du_id": "mutated in attack",
      "cu_ue_f1ap_id": "implied",
      "du_ue_f1ap_id": "implied"
    },
    "signals_or_timers": [
      "UE Context Setup Request",
      "UE Context Setup Failure",
      "UE Context Setup Response",
      "Initial UL RRC Message Transfer",
      "RRC Setup Complete"
    ]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3â€“6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

```json
{
  "candidate_spec_sections": [
    {
      "Keywords": [
        "UE Context Setup",
        "Response",
        "Failure"
      ],
      "title": [
        "F1AP 8.3.1 UE Context Setup",
        "F1AP 9.2.2.2 UE CONTEXT SETUP RESPONSE",
        "F1AP 9.2.2.3 UE CONTEXT SETUP FAILURE"
      ]
    },
    {
      "Keywords": [
        "Initial UL RRC Message Transfer",
        "RRC-Container"
      ],
      "title": [
        "F1AP 8.4.1 Initial UL RRC Message Transfer",
        "F1AP 9.2.3.1 INITIAL UL RRC MESSAGE TRANSFER"
      ]
    }
  ],
  "expected_behaviour": "The gNB-CU initiates UE Context Setup. The gNB-DU responds with UE CONTEXT SETUP RESPONSE (Success) or UE CONTEXT SETUP FAILURE. If a Failure is received or the procedure fails, the context establishment should be aborted. A subsequent Success message for the same transaction should not be accepted if the procedure has already failed or been rejected."
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

```json
{
  "candidate_code": [
    {
      "path": "openair2/F1AP/f1ap_cu_ue_context_management.c",
      "function_name": "CU_handle_UE_CONTEXT_SETUP_RESPONSE",
      "reason": "Handles the UE Context Setup Response message where the state confusion likely occurs."
    },
    {
      "path": "openair2/F1AP/f1ap_cu_ue_context_management.c",
      "function_name": "CU_handle_UE_CONTEXT_SETUP_FAILURE",
      "reason": "Handles the failure case, relevant for verifying if the context is properly cleaned up."
    },
    {
      "path": "openair2/F1AP/f1ap_cu_rrc_message_transfer.c",
      "function_name": "CU_handle_INITIAL_UL_RRC_MESSAGE_TRANSFER",
      "reason": "Handles the Initial UL RRC Message Transfer which is sent in parallel in the attack."
    }
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet â‰¤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

```json
{
  "snippets": [
    {
      "kind": "code",
      "source": "openair2/RRC/NR/rrc_gNB.c",
      "location": "2271-2282",
      "text": "static void rrc_CU_process_ue_context_setup_response(MessageDef *msg_p, instance_t instance) {\n  f1ap_ue_context_setup_resp_t *resp = &F1AP_UE_CONTEXT_SETUP_RESP(msg_p);\n  ... \n  rrc_gNB_ue_context_t *ue_context_p = rrc_gNB_get_ue_context(rrc, resp->gNB_CU_ue_id);\n  if (!ue_context_p) ... return;\n  UE->f1_ue_context_active = true;\n  ..."
    },
    {
      "kind": "code",
      "source": "openair2/RRC/NR/rrc_gNB.c",
      "location": "2046-2051",
      "text": "void rrc_gNB_process_initial_ul_rrc_message(sctp_assoc_t assoc_id, const f1ap_initial_ul_rrc_message_t *ul_rrc) {\n  ... \n  LOG_I(NR_RRC, \"Decoding CCCH: RNTI %04x...\", ul_rrc->crnti, ul_rrc->rrc_container_length);\n  ..."
    },
    {
      "kind": "spec",
      "source": "F1AP TS 38.473",
      "location": "9.2.2.2",
      "text": "UE CONTEXT SETUP RESPONSE\nThis message is sent by the gNB-DU to confirm the setup of a UE context.\nDirection: gNB-DU -> gNB-CU"
    },
    {
      "kind": "spec",
      "source": "F1AP TS 38.473",
      "location": "9.2.2.3",
      "text": "UE CONTEXT SETUP FAILURE\nThis message is sent by the gNB-DU to indicate that the setup of the UE context was unsuccessful.\nDirection: gNB-DU -> gNB-CU"
    }
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5â€“8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5â€“8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

```json
{
  "sequence_diagram": [
    {
      "from": "Attacker",
      "to": "gNB-CU",
      "message": "Initial UL RRC Message Transfer (Trigger)",
      "precond": "No Context",
      "postcond": "Context Created (Pending)"
    },
    {
      "from": "gNB-CU",
      "to": "Attacker",
      "message": "UE Context Setup Request",
      "precond": "Context Created",
      "postcond": "Awaiting Response"
    },
    {
      "from": "Attacker",
      "to": "gNB-CU",
      "message": "UE Context Setup Failure",
      "precond": "Awaiting Response",
      "postcond": "Context Cleaning/Null"
    },
    {
      "from": "Attacker",
      "to": "gNB-CU",
      "message": "UE Context Setup Response",
      "precond": "Context Cleaning/Null",
      "postcond": "Context Active (Bug)"
    }
  ],
  "state_machine": {
    "states": [
      "NULL",
      "AWAITING_RESPONSE",
      "ESTABLISHED",
      "FAILURE_HANDLING",
      "ACTIVE"
    ],
    "transitions": [
      {
        "from": "NULL",
        "to": "AWAITING_RESPONSE",
        "on": "Send Setup Req",
        "guard": "Alloc Success"
      },
      {
        "from": "AWAITING_RESPONSE",
        "to": "NULL",
        "on": "Recv Failure",
        "guard": "None"
      },
      {
        "from": "AWAITING_RESPONSE",
        "to": "ESTABLISHED",
        "on": "Recv Response",
        "guard": "Valid ID"
      },
      {
        "from": "NULL",
        "to": "ESTABLISHED",
        "on": "Recv Response",
        "guard": "ID exists (Bug)"
      }
    ]
  },
  "assumptions": [
    "The 'incorrect UE Context Setup Request' in the bug report refers to an Initial UL RRC Message or a flow that establishes a temporary context.",
    "The CU does not immediately delete the context upon failure, or the Response arrives before deletion is complete.",
    "The code does not check if the transaction is in 'AWAITING_RESPONSE' state before processing 'UE CONTEXT SETUP RESPONSE'."
  ],
  "questions": [],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

```json
{
  "invariant_checks": [
    {
      "name": "Request-Response Pairing",
      "result": "FAIL",
      "evidence": [
        "Code `rrc_CU_process_ue_context_setup_response` only checks if `ue_context_p` exists.",
        "It does not check if a Setup Request is outstanding or if the state allows a response."
      ]
    },
    {
      "name": "State Consistency",
      "result": "FAIL",
      "evidence": [
        "Bug report states context is established after failure.",
        "Code unconditionally sets `UE->f1_ue_context_active = true` upon receiving Response."
      ]
    }
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "The implementation fails to enforce that a UE Context Setup Response must strictly follow a successful UE Context Setup Request procedure and validation. It allows transition to Active state from an invalid or failed state."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

```json
{
  "evidence_pack": {
    "spec_citations": [
      "F1AP TS 38.473 8.3.1",
      "F1AP TS 38.473 9.2.2.2",
      "F1AP TS 38.473 9.2.2.3"
    ],
    "code_citations": [
      "openair2/RRC/NR/rrc_gNB.c:2271-2282",
      "openair2/RRC/NR/rrc_gNB.c:2046-2051"
    ],
    "expected_vs_observed": "Expected: UE Context Setup Response should only be processed if the CU is awaiting a response for a valid Request. Observed: CU processes Response and activates context merely because the Context ID exists, ignoring previous failures or lack of outstanding request."
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "The gNB-CU RRC layer (`rrc_gNB.c`) mishandles `UE CONTEXT SETUP RESPONSE`. The function `rrc_CU_process_ue_context_setup_response` retrieves the UE context based on ID and immediately sets `f1_ue_context_active = true`. It lacks a check to ensure the context is in a state expecting this response (e.g., following a Setup Request). This allows an attacker to bypass rejection/failure logic by injecting a Success message, leading to an unauthorized or inconsistent Active state.",
    "citations": {
      "spec": [
        "F1AP TS 38.473 Clause 8.3.1 UE Context Setup"
      ],
      "code": [
        "openair2/RRC/NR/rrc_gNB.c"
      ]
    },
    "suggested_repro": [
      "1. Send Initial UL RRC Message to create temporary context at CU.",
      "2. Send (or cause) a UE Context Setup Request that will fail or be rejected.",
      "3. Send UE CONTEXT SETUP FAILURE (or wait for rejection).",
      "4. Immediately send UE CONTEXT SETUP RESPONSE with the same IDs.",
      "5. Verify if CU considers the context Active."
    ],
    "risks": [
      "Unauthorized UE context establishment",
      "State desynchronization between CU and DU",
      "Potential Denial of Service"
    ],
    "next_steps": [
      "Add state verification in `rrc_CU_process_ue_context_setup_response`.",
      "Ensure `f1_ue_context_active` is only set if the procedure was actually successful and expected."
    ]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.